In [1]:
%load_ext autoreload

In [2]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [18]:
%autoreload
import sbert_training
from sklearn.metrics import precision_recall_fscore_support

In [4]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
from sentence_transformers import util
from zipfile import ZipFile
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.datasets import NoDuplicatesDataLoader

import logging

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [5]:
output_path = "../../data-ceph/arguana/argmining22-sharedtask/models/"

In [6]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

taska_validity_train_df = taska_training_df[taska_training_df.Validity != 0].copy()
taska_validity_valid_df = taska_valid_df[taska_valid_df.Validity != 0].copy()

taska_validity_train_df['label'] = taska_validity_train_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_validity_valid_df['label'] = taska_validity_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_novelty_train_df = taska_training_df[taska_training_df.Novelty != 0].copy()
taska_novelty_valid_df = taska_valid_df[taska_valid_df.Novelty != 0].copy()

taska_novelty_train_df['label'] = taska_novelty_train_df.Novelty.apply(lambda x : 1 if x == 1 else 0)
taska_novelty_valid_df['label'] = taska_novelty_valid_df.Novelty.apply(lambda x : 1 if x == 1 else 0)

In [7]:
taska_novelty_train_df.label.value_counts()

0    595
1    123
Name: label, dtype: int64

In [8]:
taska_validity_train_df.label.value_counts()

1    401
0    320
Name: label, dtype: int64

### Evaluate sbert on Validity:

In [9]:
n=10

In [19]:
all_f1_scores = []
for i in range(n):
    trained_model, evaluator = sbert_training.train_model(taska_validity_train_df, taska_validity_valid_df, output_path + '/task-A/validity/sbert/', 
            'sentence-transformers/nli-roberta-large', 
            num_epochs=5, train_batch_size=32,
            model_suffix='', max_seq_length=512, special_tokens=[], 
            loss='ContrastiveLoss', sentence_transformer=False, evaluation_steps=10)
    
    eval_df = sbert_training.predict_labels(taska_validity_valid_df, trained_model, 'Premise', 'Conclusion', 'pred_validity', 0.5)
    
    precision, recall, f1, _ = precision_recall_fscore_support(taska_validity_valid_df.Validity.tolist(), taska_validity_valid_df.pred_validity.tolist(), average='binary')
    print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))
    all_f1_scores.append(f1)

2022-07-04 14:55:41 - Use pytorch device: cuda
2022-07-04 14:55:41 - Read Triplet train dataset
Len of training: 721
Len of Dev: 199
Evaluating before start learning.....
2022-07-04 14:55:41 - Binary Accuracy Evaluation of the model on sts-dev dataset:
2022-07-04 14:55:41 - Accuracy with Cosine-Similarity:           72.36	(Threshold: 0.6503)
2022-07-04 14:55:41 - F1 with Cosine-Similarity:                 80.69	(Threshold: 0.5985)
2022-07-04 14:55:41 - Precision with Cosine-Similarity:          70.91
2022-07-04 14:55:41 - Recall with Cosine-Similarity:             93.60
2022-07-04 14:55:41 - Average Precision with Cosine-Similarity:  82.54

2022-07-04 14:55:41 - Accuracy with Manhatten-Distance:           72.36	(Threshold: 646.3732)
2022-07-04 14:55:41 - F1 with Manhatten-Distance:                 80.56	(Threshold: 679.7071)
2022-07-04 14:55:41 - Precision with Manhatten-Distance:          71.17
2022-07-04 14:55:41 - Recall with Manhatten-Distance:             92.80
2022-07-04 14:55:41

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

2022-07-04 14:55:43 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-04 14:55:43 - Accuracy with Cosine-Similarity:           73.37	(Threshold: 0.7017)
2022-07-04 14:55:43 - F1 with Cosine-Similarity:                 80.13	(Threshold: 0.6247)
2022-07-04 14:55:43 - Precision with Cosine-Similarity:          69.19
2022-07-04 14:55:43 - Recall with Cosine-Similarity:             95.20
2022-07-04 14:55:43 - Average Precision with Cosine-Similarity:  82.63

2022-07-04 14:55:43 - Accuracy with Manhatten-Distance:           73.87	(Threshold: 601.6650)
2022-07-04 14:55:43 - F1 with Manhatten-Distance:                 80.43	(Threshold: 625.3463)
2022-07-04 14:55:43 - Precision with Manhatten-Distance:          72.44
2022-07-04 14:55:43 - Recall with Manhatten-Distance:             90.40
2022-07-04 14:55:43 - Average Precision with Manhatten-Distance:  82.47

2022-07-04 14:55:43 - Accuracy with Euclidean-Distance:           73.37	(Threshold: 24.0499

...

Iteration:   0%|          | 0/23 [00:00<?, ?it/s]

2022-07-04 15:02:07 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 10 steps:
2022-07-04 15:02:07 - Accuracy with Cosine-Similarity:           72.86	(Threshold: 0.7045)
2022-07-04 15:02:07 - F1 with Cosine-Similarity:                 80.13	(Threshold: 0.6373)
2022-07-04 15:02:07 - Precision with Cosine-Similarity:          69.19
2022-07-04 15:02:07 - Recall with Cosine-Similarity:             95.20
2022-07-04 15:02:07 - Average Precision with Cosine-Similarity:  82.22

2022-07-04 15:02:07 - Accuracy with Manhatten-Distance:           72.86	(Threshold: 616.0450)
2022-07-04 15:02:07 - F1 with Manhatten-Distance:                 80.71	(Threshold: 616.0450)
2022-07-04 15:02:07 - Precision with Manhatten-Distance:          72.90
2022-07-04 15:02:07 - Recall with Manhatten-Distance:             90.40
2022-07-04 15:02:07 - Average Precision with Manhatten-Distance:  82.26

2022-07-04 15:02:07 - Accuracy with Euclidean-Distance:           72.36	(Threshold: 23.1337

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Precision: 0.6410256410256411, Recall 1.0, F1: 0.7812500000000001


In [22]:
print('Average F1-score: {}'.format(f1))

Average F1-score: 0.7812500000000001


### Evaluate sbert on Novelty:

In [ ]:
all_f1_scores = []
for i in range(n):
    trained_model, evaluator = sbert_training.train_model(taska_validity_train_df, taska_validity_valid_df, output_path + '/task-A/validity/sbert/', 
            'sentence-transformers/nli-roberta-large', 
            num_epochs=5, train_batch_size=32,
            model_suffix='', max_seq_length=512, special_tokens=[], 
            loss='ContrastiveLoss', sentence_transformer=False, evaluation_steps=10)
    
    eval_df = sbert_training.predict_labels(taska_validity_valid_df, trained_model, 'Premise', 'Conclusion', 'pred_validity', 0.5)
    
    precision, recall, f1, _ = precision_recall_fscore_support(taska_validity_valid_df.Validity.tolist(), taska_validity_valid_df.pred_validity.tolist(), average='binary')
    print('Precision: {}, Recall {}, F1: {}'.format(precision, recall, f1))
    all_f1_scores.append(f1)